In [6]:
from dotenv import load_dotenv
import streamlit as st
import os
import google.generativeai as genai

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Will house our FAISS vector store
store = None

# Will convert text into vector embeddings using OpenAI.
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

def split_paragraphs(rawText):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )

    return  text_splitter.split_text(rawText)

def load_pdfs(pdfs):
    text_chunks = []

    for pdf in pdfs:
        reader = PdfReader(pdf)
        for page in reader.pages:
            raw = page.extract_text()
            chunks = split_paragraphs(raw)
            text_chunks += chunks
    return text_chunks


In [7]:
def main():
    list_of_pdfs = ["rule11.pdf", "rule1.pdf", "rule2.pdf", "rule3.pdf", "rule4.pdf", "rule5.pdf", "rule6.pdf", "rule7.pdf", "megapdf.pdf","rule13.pdf"]
    text_chunks = load_pdfs(list_of_pdfs)
    # Index the text chunks in our FAISS store.
    # OpenAIEmbeddings will be used automatically to convert
    # Each chunk of text into vector embeddings using
    # OpenAI APIs. 
    store = FAISS.from_texts(text_chunks, embeddings)

    # Write our index to disk.
    store.save_local("./vectorstore2")

if __name__ == "__main__":
    main()
# Tell me about the arrangement of clauses in the bharatiya nyaya sanhita